This notebook is freely available for redistribution under the [GPL-3.0 license](https://choosealicense.com/licenses/gpl-3.0/).

Author: 蘇嘉冠

# [Exercise 2] Regression

## 展示題：房價預測（單變數版本）

我們蒐集到了波士頓郊區的房價資料集（[來源](https://archive.ics.uci.edu/ml/machine-learning-databases/housing/?C=N;O=D)），想要從某城鎮的一個 feature，來預測該城鎮自用住宅的房價中位數（`MEDV`）。

我們將這個資料集的 csv 檔讀入至一個 pandas 的 DataFrame：`df`。資料的各個 column 的意義如下：
- `CRIM`：某城鎮的人均犯罪率
- `ZN`：「超過 25,000 平方呎的住宅用地區塊」所佔的比例
- `INDUS`：某城鎮「非零售的商業用地」比例（英畝）
- `NOX`：一氧化氮濃度（以 10 ppm 為單位）
- `RM`：平均每戶有幾個房間
- `AGE`： 1940 年之前所建的房屋，屋主自用的比例
- `DIS`：到波士頓五個就業服務中心的（加權）距離
- `RAD`：使用高速公路的方便性 / 可達性指數
- `TAX`：「總價 / 房屋稅」的比例（單位：10,000 美金）
- `PTRATIO`：某城鎮的「生 / 師」比
- `LSTAT`：低所得的人口比例
- `MEDV`：自用住宅的房價中位數（單位：1,000 美金）




In [ ]:
!pip install numpy pandas matplotlib scikit-learn mlxtend==0.18.0

### Exploratory Data Analysis (EDA)

在開使訓練一個機器學習模型之前，一個很重要的工作是做 Exploratory Data Analysis (EDA)，透過視覺化的方式來了解資料的分佈、feature 之間的關係等。

In [3]:
import pandas as pd

df = pd.read_csv(
    "https://raw.githubusercontent.com/AINTUT/code_2022/main/datasets/"
    "house_pricing.csv",
)

In [ ]:
print(df.head())

In [ ]:
print(list(df))

第一步是用各個 feature 之間的分佈來看 feature 之間的關係。在這裡我們使用 [mlxtend](https://rasbt.github.io/mlxtend/) 這個 package，並且選了幾個 feature 來做視覺化，可以嘗試看看修改 `columns` 的內容來看不同的 feature，或是將 `columns` 改成 `columns = list(df)` 來看所有 feature 之間的關係。

In [ ]:
import matplotlib.pyplot as plt
from mlxtend.plotting import scatterplotmatrix

columns = [
    "CRIM",
    "NOX",
    "RM",
    "DIS",
    "TAX",
    "PTRATIO",
    "MEDV",
]

scatterplotmatrix(
    df[columns].to_numpy(),
    figsize=(10, 8),
    names=columns,
    alpha=0.5,
)

plt.tight_layout()
plt.show()

第二步是去算各個 feature 之間的 [correlation coefficient](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient)，並且用熱力圖（heatmap）來做呈現。這個數值如果接近 1 代表某兩個 feature 有很強的正關係，接近 -1 代表有很強的負關係，接近 0 則代表關係不大。

In [ ]:
import numpy as np
from mlxtend.plotting import heatmap

cm = np.corrcoef(df[columns].to_numpy().T)
hm = heatmap(cm, row_names=columns, column_names=columns)

plt.show()

做完以上的 EDA 之後，我們認為 `RM` 這個變數跟我們的目標：房價（`MEDV`）很有關係，所以將這兩者之間的散佈圖畫出來一下，並且將 `RM` 作為 input feature（`x_data`），`MEDV` 作為預測的目標（`y_data`）。

附註：
- `x_data` 的 shape 為 (506, 1)，前面的數字代表有 506 筆資料，後面的 1 則代表有一個 input feature。之後我們學到多個 input feature 時，後面的數字會大於 1。
- `y_data` 的 shape (506,)，只有一個數字，因為每筆資料對應到的目標數值永遠只有一個，所以不需要再額外多一個維度。

In [ ]:
df.plot.scatter(x="RM", y="MEDV")
plt.show()

In [ ]:
x_data = df[["RM"]].to_numpy()
y_data = df["MEDV"].to_numpy()

print(x_data.shape)
print(y_data.shape)

### Data Preprocessing

選定 feature 後，我們將對資料做兩個步驟的前處理

第一步，使用 [scikit-learn](https://scikit-learn.org/stable/index.html) 的 [train_test_split()](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)，將資料分割成 training data 與 testing data，比例為 80:20。

附註：
- `x_train` 與 `y_train` 代表 training data
- `x_test` 與 `y_test` 代表 testing data

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
    x_data,
    y_data,
    test_size=0.2,
    random_state=0,
)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

第二步，使用 scikit-learn 的 [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html)，以 standardization 的方法對資料做 feature scaling。在做 standardization 需要給一批資料來算這批資料的平均值與標準差，然後在實際的場景中，我們並不會知道 testing data 的平均值與標準差。因此我們的作法是：先算出 training data 的平均值與標準差，將之同時用在 training 與 testing data 上。

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

x_train_std = scaler.fit_transform(x_train)
x_test_std = scaler.transform(x_test)

print("Before standardization:")
print(x_train[:10])
print("After standardization:")
print(x_train_std[:10])

### Training

附註：

這個段落比較困難，數學的部份可以跳過，有興趣挑戰的同學再來看細節。因為目前很多工具都有實做類似的功能，以應用的角度來說只要知道主要觀念（訓練的三個大步驟）以及如何使用就好。

開始我們訓練模型的流程，主要包含幾個步驟：
1. 定義 function set：$f(x) = b + w_{1}x_{1}$
2. 定義 loss function：$L(b, w_{1}) = \frac{1}{2}\sum_{i=1}^{n}(\hat{y}^{(i)} - (b + w_{1}{x}^{(i)}_{1}))^2$
3. 學習，使用 Gradient Descent 來找最佳的 function：$f^{*}$

用 `predict()` 這個 function 來實現 $f(x)$ 的計算過程，其中：
- `x` 代表某 n 筆房屋的資料，型別為 `numpy.ndarray`，如果每筆資料有 k 個 input feature，則 `x` 的 shape 為 (n, k) （目前我們只有考慮 `RM` 這個 feature，也就是 k = 1，而 `predict()` 可以同時處理 k > 1 的情況 ）
- `weights` 代表權重，型別為 `numpy.ndarray`。目前我們的權重會長的像 $[b, w_{1}]$ 這樣，`predict()` 可以處理擴展到 k + 1 個權重的狀況（$[b, w_1, w_2, ..., w_k]$）

根據 `x` 跟 `weights`，我們可以算出相對應的 $f(x)$ 。例如我們 k = 1，有 3 筆資料的時候，我們可以算出這 3 筆資料相對應的結果：
$f(x) = \begin{bmatrix}x^{(1)}_{1} \\ x^{(2)}_1 \\ x^{(3)}_{1} \end{bmatrix}\dot{}\begin{bmatrix}w_{1}\end{bmatrix} + b = \begin{bmatrix}w_{1}x^{(1)}_{1} \\ w_{1}x^{(2)}_1 \\ w_{1}x^{(3)}_{1} \end{bmatrix} + b = \begin{bmatrix}w_{1}x^{(1)}_{1} + b \\ w_{1}x^{(2)}_1 + b \\ w_{1}x^{(3)}_{1} + b \end{bmatrix}$

In [25]:
def predict(x, weights):
    return np.dot(x, weights[1:]) + weights[0]

用 `calculate_loss()` 這個 function 計算 loss：$L(b, w_{1})$（SSE, Sum of Squared Error），其中：
- `y_gt`：代表某 n 筆房屋真實的房價，型別為 `numpy.ndarray`，shape 為 (n,)
- `y_pred`：代表某 n 筆房屋預測的房價，型別為 `numpy.ndarray`，shape 為 (n,)

根據 `y_gt` 與 `y_pred`，來算出 loss 的數值。

In [26]:
def calculate_loss(y_gt, y_pred):
    loss = ((y_gt - y_pred) ** 2).sum() / 2.0

    return loss

`fit()` 這個 function 用來跑訓練的流程：
1. 初始化 `weights`，數值皆為 0（我們只有 1 個 feature，所以 `weights` 長的像這樣：$[0, 0]$）
2. 做 gradient descent：先根據目前的 `weights` 來計算 `f(x)`，再計算 `weights` 中每個值對 loss 的偏微分，並且用偏微分的結果來更新 `weights`（更新的幅度由 `learning_rate` 來決定）
3. 計算 loss 並且記錄下來
4. 重複步驟 2 ~ 3 好幾個 epoch（由 `epoches` 來決定要重複幾次）
5. 訓練完成後，將訓練好的 `weights`，以及每個 epoch 紀錄下來的 loss 回傳回去

關於偏微分請參考課程的[簡報](https://docs.google.com/presentation/d/1Dntz67pZ_mbShQZhkyO9llqcx7pv7Pbs_R67Lf-5UGQ/edit#slide=id.gcd0121f0ef_0_1)。`fit()` 可以處理有 k 個 feature 的情況，而簡報是在說 k = 1，也就是我們目前資料的狀況。


In [27]:
def fit(x_train, y_train, epoches, learning_rate):
    weights = np.zeros(x_train.shape[1] + 1)
    losses = []

    for _ in range(epoches):
        y_pred = predict(x_train, weights)

        diff = y_train - y_pred
        weights[0] = weights[0] - learning_rate * -diff.sum()
        weights[1:] = weights[1:] - learning_rate * -x_train.T.dot(diff)

        losses.append(calculate_loss(y_train, y_pred))

    return weights, losses

開始跑我們的訓練，設定好 `epoches` 以及 `learning_rate` 這兩個 hyper parameter，並且將訓練資料與 hyper parameters 丟進去 `fit()`，得到訓練好的 `weights` 以及 `losses`（每個 epoch 紀錄下來的 loss）

In [28]:
epoches = 20
learning_rate = 0.001

weights, losses = fit(x_train_std, y_train, epoches, learning_rate)

我們將每個 epoch 的 loss 的圖畫出來，來確認 loss 是否有逐漸下降！

In [ ]:
plt.plot(range(1, epoches + 1), losses)
plt.ylabel("SSE")
plt.xlabel("Epoch")
plt.show()

### Evaluation

來看一下我們的訓練成果！

這裡的 `reg_plot()` 將以下資訊同時畫在同張圖：
1. 每筆房屋資料，input feature 與真實房價之間的散佈圖
2. 每筆房屋資料，input feature 與預測房價之間的曲線圖

In [30]:
def reg_plot(x, y_gt, y_pred):
    plt.scatter(x, y_gt, c="steelblue", edgecolor="white")
    plt.plot(x, y_pred, c="black")

    plt.xlabel("RM")
    plt.ylabel("MEDV")

    plt.show()

對於 training data，先用 `reg_plot()` 畫出結果的圖來，再用 scikit-learn 的 [mean_squared_error()](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html?highlight=mean_squared_error#sklearn.metrics.mean_squared_error) 算 Mean Squared Error（MSE）的數值

In [ ]:
from sklearn.metrics import mean_squared_error

y_pred = predict(x_train_std, weights)

reg_plot(x_train, y_train, y_pred)

mse = mean_squared_error(y_train, y_pred)

print("MSE of Training Data: {}".format(mse))

對於 testing data，先用 `reg_plot()` 畫出結果的圖來，再算 Mean Squared Error（MSE）的數值。

附註：一般而言，testing data 的 MSE 會比 training 的還大

In [ ]:
y_pred = predict(x_test_std, weights)

reg_plot(x_test, y_test, y_pred)

mse = mean_squared_error(y_test, y_pred)

print("MSE of Testing Data: {}".format(mse))

### 改用 Scikit-Learn 做 Training

如果不想要實做上面複雜的 training 流程，我們可以改用 scikit-learn 的 [LinearRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html?highlight=linearregression#sklearn.linear_model.LinearRegression) 來實做，可以在很少行 code 的狀況下完成一個 linear regression 的訓練。不過要注意的是 scikit-learn 並不是用 gradient descent 來實做（[參考](https://stackoverflow.com/questions/34469237/linear-regression-and-gradient-descent-in-scikit-learn)），而且不用對資料做 feature scaling。

In [ ]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(x_train, y_train)

In [ ]:
y_pred = lr.predict(x_train)

reg_plot(x_train, y_train, y_pred)

mse = mean_squared_error(y_train, y_pred)

print("MSE of Training Data: {}".format(mse))

In [ ]:
y_pred = lr.predict(x_test)

reg_plot(x_test, y_test, y_pred)

mse = mean_squared_error(y_test, y_pred)

print("MSE of Testing Data: {}".format(mse))

## 練習題

1. 我們想改用 `LSTAT` 作為 input feature，來預測房價（`MEDV`）。請修改上面的程式碼來達成我們的目標，並觀察 training 與 testing data 的 MSE 為多少。
2. 我們想改用 `CRIM` 作為 input feature，來預測房價（`MEDV`）。請修改上面的程式碼來達成我們的目標，並觀察 training 與 testing data 的 MSE 為多少。
3. 上面兩者的 MSE，何者比較低？造成這種結果的原因是什麼